In [ ]:
import torch
from torch import nn
import torch.optim as optim
from torchvision import transforms
from torchvision.datasets import MNIST # Training dataset
from torchvision.utils import make_grid
from torch.utils.data import DataLoader
from torch.utils.data.sampler import SubsetRandomSampler
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
# obtain mnist data and process
batch_size = 128
num_of_classes = 10

def get_indices(dataset,ind_array):
    indices =  []
    for i in range(len(dataset.targets)):
        for ind in ind_array:
            if dataset.targets[i] == ind:
                indices.append(i)
    return indices

dataset = MNIST('.', train=True, download=False,transform=transforms.ToTensor())

idx = get_indices(dataset, np.arange(num_of_classes))
data_loader_target = DataLoader(dataset, batch_size=batch_size, sampler = SubsetRandomSampler(idx))
data_loader_gan = DataLoader(dataset, batch_size=batch_size, sampler = SubsetRandomSampler(idx))

classes = np.arange(num_of_classes)

ims = []
lls = []
for imgs,lbls in data_loader_target:
    for img,lbl in zip(imgs,lbls):
        ims.append(img)
        lls.append(lbl)
ims = torch.stack(ims)
lls = torch.stack(lls)

In [ ]:
%%time 

# target model
from net_conv import target_net

if torch.cuda.is_available():  
    dev = 'cuda:0'
else:  
    dev = 'cpu'

print('device: ', dev)
print('-----------------------------')

from sklearn.ensemble import RandomForestClassifier
# target model
n_estimators = 500
rf = RandomForestClassifier(n_estimators=n_estimators)
rf.fit(ims.reshape(len(ims),28*28).detach().numpy(),lls)
from joblib import dump, load
# save the model
PATH = './target_models/rf_model'+str(num_of_classes)+'classes'+'_nest'+str(n_estimators)+'.joblib'
dump(rf, PATH) 
# load the model
rf = load(PATH)

# accuracy of target model
correct = 0.
total = 0.
for img,lbl in data_loader_gan:
    outputs = rf.predict(img.reshape(len(img),28*28).detach().numpy())
    total += lbl.size(0)
    correct += sum(outputs == lbl.detach().numpy())
print('accuracy: ', correct/total)

In [ ]:
from net_ import target_net
net = target_net(num_of_classes)
criterion_tar = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)

# load the model
net.train(data_loader_target, criterion_tar, optimizer,master_model=rf.predict)
PATH = './target_models/rfDistilled'+str(num_of_classes)+'classes_'+str(n_estimators)+'nest.pth'
torch.save(net.state_dict(), PATH)
net.load_state_dict(torch.load(PATH))

print('model accuracy: ', net.accuracy(data_loader_gan))

In [ ]:
%%time 
from advGAN import advGAN
advgan = advGAN(net,gen_path_extra='rfDistilled',device=dev)

In [ ]:
gen,disc = advgan.train(data_loader_gan)

In [ ]:
# load the generator 
gen = advgan.load_gen()

In [ ]:
from sklearn.metrics import accuracy_score
# test transferability from adv examples for the nn to the rf
counter = 0.
total_rf_clean = 0.
total_nn_clean = 0.
total_rf_adv = 0.
total_nn_adv = 0.
for data,label in DataLoader(dataset, batch_size=batch_size, sampler = SubsetRandomSampler(idx)):
    rf_output_clean = rf.predict(data.reshape(len(data),28*28).detach().numpy())
    nn_output_clean = torch.argmax(net(data.reshape(len(data),28*28)),dim=1)
    # make adv example
    pert = gen(data.reshape(len(data),28*28))
    adv_img = data.reshape(len(data),28*28) + pert
    rf_output_adv = rf.predict(adv_img.detach().numpy())
    nn_output_adv = torch.argmax(net(adv_img),dim=1)
    # calculate and return accuracy 
    total_rf_clean += accuracy_score(rf_output_clean,label)
    total_nn_clean += accuracy_score(nn_output_clean,label)
    total_rf_adv += accuracy_score(rf_output_adv,label)
    total_nn_adv += accuracy_score(nn_output_adv,label)
    counter += 1.
print('rf accuracy (clean): ', total_rf_clean/counter)
print('nn accuracy (clean): ', total_nn_clean/counter)
print('% adv rf: ', 1.-total_rf_adv/counter)
print('% adv nn: ', 1.-total_nn_adv/counter)